In [0]:
%pip install -r requirements.lock

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import nest_asyncio
nest_asyncio.apply()

In [0]:
import fsspec
import glob
import json
import os

import xarray as xr

from pathlib import Path

from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr

Create Kerchunk references for multiple NetCDF files.

In [0]:
%sh python create_kerchunk_refs.py "/tmp/benchmarking/*.nc" /tmp/benchmarking/references/


NETCDF → KERCHUNK REFERENCE GENERATION
Start time: 2025-12-03 17:41:21

KERCHUNK REFERENCE GENERATION
Input pattern: /tmp/benchmarking/*.nc
Files found: 100
Output directory: /tmp/benchmarking/references
Concat dimensions: ['valid_time']
Identical dimensions: ['number', 'latitude', 'longitude']

----------------------------------------------------------------------
Step 1: Creating individual reference files...
----------------------------------------------------------------------
  ✓ Created 10/100 references...
  ✓ Created 20/100 references...
  ✓ Created 30/100 references...
  ✓ Created 40/100 references...
  ✓ Created 50/100 references...
  ✓ Created 60/100 references...
  ✓ Created 70/100 references...
  ✓ Created 80/100 references...
  ✓ Created 90/100 references...
  ✓ Created 100/100 references...

  Created 100 reference files in 1.53s

----------------------------------------------------------------------
Step 2: Combining references into master.json...
---------------------

In [0]:
%sh python create_kerchunk_refs.py "/Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc" /Volumes/stuart/lseg/netcdf/weather/benchmarking/references


NETCDF → KERCHUNK REFERENCE GENERATION
Start time: 2025-12-03 18:10:12

KERCHUNK REFERENCE GENERATION
Input pattern: /Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc
Files found: 100
Output directory: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references
Concat dimensions: ['valid_time']
Identical dimensions: ['number', 'latitude', 'longitude']

----------------------------------------------------------------------
Step 1: Creating individual reference files...
----------------------------------------------------------------------
  ✓ Created 10/100 references...
  ✓ Created 20/100 references...
  ✓ Created 30/100 references...
  ✓ Created 40/100 references...
  ✓ Created 50/100 references...
  ✓ Created 60/100 references...
  ✓ Created 70/100 references...
  ✓ Created 80/100 references...
  ✓ Created 90/100 references...
  ✓ Created 100/100 references...

  Created 100 reference files in 280.32s

----------------------------------------------------------------------
Step 2

In [0]:
%sh python benchmark_loading_kerchunk.py \
  "/tmp/benchmarking/references/*.json" \
  -o local/


XARRAY KERCHUNK LOADING BENCHMARK
Reference path: /tmp/benchmarking/references/*.json
Mode: Individual JSON references
Remote protocol: file
Number of reference files: 100
Start time: 2025-12-03 17:45:14


Test 1/3

Test: Individual JSONs, native chunks
  Path: /tmp/benchmarking/references/*.json
  Mode: Individual JSON references
  Chunks: None
  Remote protocol: file
  Found 100 reference files
✓ Opened dataset in 17.405 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: None
✓ Accessed sample data in 0.001 seconds
  Sample value: 314.65
✓ Total time: 17.406 seconds


Test 2/3

Test: Individual JSONs, auto rechunking
  Path: /tmp/benchmarking/references/*.json
  Mode: Individual JSON references
  Chunks: {}
  Remote protocol: file
  Found 100 reference files
✓ Opened dataset in 1.395 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: Fro

In [0]:
%sh python benchmark_loading_kerchunk.py \
  --combined \
  "/tmp/benchmarking/references/master.json" \
  -o local/


XARRAY KERCHUNK LOADING BENCHMARK
Reference path: /tmp/benchmarking/references/master.json
Mode: Combined master.json
Remote protocol: file
Start time: 2025-12-03 17:47:04


Test 1/4

Test: Combined master.json, native chunks
  Path: /tmp/benchmarking/references/master.json
  Mode: Combined master.json
  Chunks: None
  Remote protocol: file
✓ Opened dataset in 0.240 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: None
✓ Accessed sample data in 0.026 seconds
  Sample value: 314.65
✓ Total time: 0.267 seconds


Test 2/4

Test: Combined master.json, auto rechunking
  Path: /tmp/benchmarking/references/master.json
  Mode: Combined master.json
  Chunks: {}
  Remote protocol: file
✓ Opened dataset in 0.095 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: Frozen({'number': (50,), 'valid_time': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [0]:
%sh python benchmark_loading_kerchunk.py \
  "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/*.json" \
  -o remote/

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:190)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:201)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
%sh python benchmark_loading_kerchunk.py \
  --combined \
  "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json" \
  -o remote/


XARRAY KERCHUNK LOADING BENCHMARK
Reference path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/*.json
Mode: Individual JSON references
Remote protocol: file
Number of reference files: 100
Start time: 2025-12-03 17:48:49


Test 1/3

Test: Individual JSONs, native chunks
  Path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/*.json
  Mode: Individual JSON references
  Chunks: None
  Remote protocol: file
  Found 100 reference files
✓ Opened dataset in 479.727 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: None
✓ Accessed sample data in 0.001 seconds
  Sample value: 288.62
✓ Total time: 479.728 seconds


Test 2/3

Test: Individual JSONs, auto rechunking
  Path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/*.json
  Mode: Individual JSON references
  Chunks: {}
  Remote protocol: file
  Found 100 reference files


Traceback (most recent call last):
  File "/Workspace/Users/stuart.lynn@databricks.com/customers/LSEG/NetCDF/xarray-netcdf-benchmarking/benchmark_loading_kerchunk.py", line 546, in <module>
    exit(main())
         ^^^^^^
  File "/Workspace/Users/stuart.lynn@databricks.com/customers/LSEG/NetCDF/xarray-netcdf-benchmarking/benchmark_loading_kerchunk.py", line 531, in main
    results = run_benchmarks(
              ^^^^^^^^^^^^^^^
  File "/Workspace/Users/stuart.lynn@databricks.com/customers/LSEG/NetCDF/xarray-netcdf-benchmarking/benchmark_loading_kerchunk.py", line 359, in run_benchmarks
    result = benchmark_kerchunk_loading(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Workspace/Users/stuart.lynn@databricks.com/customers/LSEG/NetCDF/xarray-netcdf-benchmarking/benchmark_loading_kerchunk.py", line 169, in benchmark_kerchunk_loading
    ds = open_kerchunk_multiple(
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Workspace/Users/stuart.lynn@databricks.com/customers/LSEG/NetCDF/xarray-net


XARRAY KERCHUNK LOADING BENCHMARK
Reference path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json
Mode: Combined master.json
Remote protocol: file
Start time: 2025-12-03 17:58:58


Test 1/4

Test: Combined master.json, native chunks
  Path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json
  Mode: Combined master.json
  Chunks: None
  Remote protocol: file
✓ Opened dataset in 1.599 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Chunks: None
✓ Accessed sample data in 3.452 seconds
  Sample value: 288.62
✓ Total time: 5.051 seconds


Test 2/4

Test: Combined master.json, auto rechunking
  Path: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json
  Mode: Combined master.json
  Chunks: {}
  Remote protocol: file
✓ Opened dataset in 1.617 seconds
  Dataset shape: {'number': 50, 'valid_time': 100, 'latitude': 361, 'longitude': 720}
  Variables: ['t2m']
  Ch

In [0]:
dbutils.notebook.exit("0")

In [0]:
def create_references(nc_pattern, output_dir='references'):
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    nc_files = sorted(glob.glob(nc_pattern))
    print(f"Processing {len(nc_files)} NetCDF files")
    
    reference_files = []
    
    # Create individual references
    for i, nc_file in enumerate(nc_files, 1):
        ref_file = output_dir / f"{Path(nc_file).stem}.json"
        
        with fsspec.open(nc_file, mode='rb') as f:
            h5chunks = SingleHdf5ToZarr(f, nc_file, inline_threshold=100)
            refs = h5chunks.translate()
        
        with open(ref_file, 'w') as f:
            json.dump(refs, f)
        
        reference_files.append(str(ref_file))
        
        if i % 10 == 0:
            print(f"  Created {i}/{len(nc_files)} references...")
    
    print(f"  Created {len(reference_files)} reference files")
    
    # Combine references
    print("\nCombining references...")
    mzz = MultiZarrToZarr(
        reference_files,
        concat_dims=['valid_time'],
        identical_dims=['number', 'latitude', 'longitude'],
        coo_map={'valid_time': 'INDEX'}
    )
    
    combined_refs = mzz.translate()
    
    # Save combined reference
    combined_file = output_dir / 'master.json'
    with open(combined_file, 'w') as f:
        json.dump(combined_refs, f)
    
    print(f"Combined reference saved to: {combined_file}")
    
    return combined_file

In [0]:
create_references(
    "/tmp/benchmarking/*.nc",
    "/tmp/references",
)

Processing 100 NetCDF files
  Created 10/100 references...
  Created 20/100 references...
  Created 30/100 references...
  Created 40/100 references...
  Created 50/100 references...
  Created 60/100 references...
  Created 70/100 references...
  Created 80/100 references...
  Created 90/100 references...
  Created 100/100 references...
  Created 100 reference files

Combining references...
Combined reference saved to: /tmp/references/master.json


PosixPath('/tmp/references/master.json')

In [0]:
create_references(
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc",
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references",
)

Processing 100 NetCDF files
  Created 10/100 references...
  Created 20/100 references...
  Created 30/100 references...
  Created 40/100 references...
  Created 50/100 references...
  Created 60/100 references...
  Created 70/100 references...
  Created 80/100 references...
  Created 90/100 references...
  Created 100/100 references...
  Created 100 reference files

Combining references...
Combined reference saved to: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json


PosixPath('/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json')

In [0]:
%fs ls /Volumes/stuart/lseg/netcdf/weather/benchmarking/references

In [0]:
%sh cat /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json

In [0]:
def open_kerchunk_dataset(reference_file):
    """Open dataset from Kerchunk reference."""
    print(f"\nOpening dataset from reference: {reference_file}")

    with open(reference_file) as f:
        refs = json.load(f)

    fs = fsspec.filesystem(
        "reference",
        fo=refs,
        remote_protocol="local",
    )

    mapper = fs.get_mapper("")

    ds = xr.open_dataset(mapper, engine="zarr", backend_kwargs={"consolidated": False})

    print(f"  Dataset opened successfully!")
    print(f"  Dimensions: {dict(ds.sizes)}")
    print(f"  Variables: {list(ds.data_vars)}")

    return ds

In [0]:
ds = open_kerchunk_dataset("/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json")
ds

In [0]:
ds.info()

xarray.Dataset {
dimensions:
	valid_time = 100 ;
	latitude = 361 ;
	longitude = 720 ;
	number = 50 ;

variables:
	datetime64[ns] valid_time(valid_time) ;
		valid_time:long_name = time ;
		valid_time:standard_name = time ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	int64 number(number) ;
		number:long_name = ensemble member numerical id ;
		number:units = 1 ;
		number:standard_name = realization ;
	float32 t2m(number, valid_time, latitude, longitude) ;
		t2m:GRIB_paramId = 167 ;
		t2m:GRIB_dataType = an ;
		t2m:GRIB_numberOfPoints = 259920 ;
		t2m:GRIB_typeOfLevel = surface ;
		t2m:GRIB_stepUnits = 1 ;
		t2m:GRIB_stepType = instant ;
		t2m:GRIB_gridType = regular_ll ;
		t2m:GRIB_uvRelativeToGrid = 0 ;
		t2m:GRI

In [0]:
ds.t2m[-1][-1][-1].values

array([258.09497, 269.8234 , 278.78317, 292.10355, 260.3426 , 288.24457,
       231.16801, 241.9859 , 297.5479 , 243.32307, 285.65073, 262.16055,
       306.5065 , 257.3265 , 267.6651 , 281.25192, 313.25488, 257.95456,
       277.07095, 253.82823, 265.35147, 232.75746, 244.94478, 304.6257 ,
       303.8453 , 304.65216, 293.10803, 267.75433, 256.49646, 276.96014,
       301.16568, 311.07062, 263.88986, 318.3675 , 224.51628, 227.3476 ,
       286.44748, 235.21408, 306.64487, 272.29102, 271.37976, 269.70648,
       270.0126 , 311.05768, 268.86087, 258.5799 , 311.6846 , 298.98016,
       232.676  , 266.10165, 223.17569, 233.39348, 299.62283, 304.24155,
       249.06187, 262.67648, 304.25342, 241.27763, 281.44867, 272.3404 ,
       277.0518 , 224.49066, 241.05273, 254.22539, 288.36472, 295.5889 ,
       287.15457, 259.36438, 259.06046, 287.12708, 278.11383, 316.01962,
       241.83145, 242.04077, 224.95616, 311.17538, 255.7322 , 308.4316 ,
       279.04736, 238.98166, 233.4351 , 239.02121, 

## Option
Use Spark to generate the references before collecting and combining.

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as ST

In [0]:
def create_references_spark(nc_pattern, output_dir):

    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    root = Path(nc_pattern).parent
    filter_suffix = Path(nc_pattern).suffix

    nc_files_df = (
      spark.createDataFrame(dbutils.fs.ls(f"{root}"))
      .where(F.lower(F.col("path")).like(f"%{filter_suffix.lower()}"))
      .withColumn("path", F.regexp_replace(F.col("path"), "dbfs:/", "/"))
      )

    @F.udf(ST.StringType())
    def h5translate(path):
      with fsspec.open(path, mode='rb') as f:
        h5chunks = SingleHdf5ToZarr(f, path, inline_threshold=100)
        refs = h5chunks.translate()
      return json.dumps(refs)

    refs_df = (
      nc_files_df
      .withColumn("refs", h5translate(F.col("path")))
    )

    reference_dicts = [json.loads(r.refs) for r in (
      refs_df
      .select(F.col("refs"))
      .collect()
    )]

    print(f"  Created {len(reference_dicts)} reference sets")
    
    # Combine references
    print("\nCombining references...")
    mzz = MultiZarrToZarr(
        reference_dicts,
        concat_dims=['valid_time'],
        identical_dims=['number', 'latitude', 'longitude'],
        coo_map={'valid_time': 'INDEX'}
    )
    
    combined_refs = mzz.translate()
    
    # Save combined reference
    combined_file = output_dir / 'master.json'
    with open(combined_file, 'w') as f:
        json.dump(combined_refs, f)
    
    print(f"Combined reference saved to: {combined_file}")
    
    return combined_file

In [0]:
create_references_spark(
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc",
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references",
)

  Created 100 reference sets

Combining references...
Combined reference saved to: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json


PosixPath('/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json')

In [0]:
ds = open_kerchunk_dataset("/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json")
ds

In [0]:
ds.info()

xarray.Dataset {
dimensions:
	valid_time = 100 ;
	latitude = 361 ;
	longitude = 720 ;
	number = 50 ;

variables:
	datetime64[ns] valid_time(valid_time) ;
		valid_time:long_name = time ;
		valid_time:standard_name = time ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	int64 number(number) ;
		number:long_name = ensemble member numerical id ;
		number:units = 1 ;
		number:standard_name = realization ;
	float32 t2m(number, valid_time, latitude, longitude) ;
		t2m:GRIB_paramId = 167 ;
		t2m:GRIB_dataType = an ;
		t2m:GRIB_numberOfPoints = 259920 ;
		t2m:GRIB_typeOfLevel = surface ;
		t2m:GRIB_stepUnits = 1 ;
		t2m:GRIB_stepType = instant ;
		t2m:GRIB_gridType = regular_ll ;
		t2m:GRIB_uvRelativeToGrid = 0 ;
		t2m:GRI

In [0]:
ds.t2m[-1][-1][-1].values

array([258.09497, 269.8234 , 278.78317, 292.10355, 260.3426 , 288.24457,
       231.16801, 241.9859 , 297.5479 , 243.32307, 285.65073, 262.16055,
       306.5065 , 257.3265 , 267.6651 , 281.25192, 313.25488, 257.95456,
       277.07095, 253.82823, 265.35147, 232.75746, 244.94478, 304.6257 ,
       303.8453 , 304.65216, 293.10803, 267.75433, 256.49646, 276.96014,
       301.16568, 311.07062, 263.88986, 318.3675 , 224.51628, 227.3476 ,
       286.44748, 235.21408, 306.64487, 272.29102, 271.37976, 269.70648,
       270.0126 , 311.05768, 268.86087, 258.5799 , 311.6846 , 298.98016,
       232.676  , 266.10165, 223.17569, 233.39348, 299.62283, 304.24155,
       249.06187, 262.67648, 304.25342, 241.27763, 281.44867, 272.3404 ,
       277.0518 , 224.49066, 241.05273, 254.22539, 288.36472, 295.5889 ,
       287.15457, 259.36438, 259.06046, 287.12708, 278.11383, 316.01962,
       241.83145, 242.04077, 224.95616, 311.17538, 255.7322 , 308.4316 ,
       279.04736, 238.98166, 233.4351 , 239.02121, 